# Benchmark ML Computation Speed

In this notebook, we test the computational performance of [digifellow](https://digifellow.swfcloud.de/hub/spawn) jupyterhub performance against free access like *Colab* and *Kaggle*. The baseline of this comparison is an average PC *(Core i5 2.5GHz - 8GB RAM - No GPU)* 

The task of this test is classifying the MNIST dataset with different algorithms *(LR, ANN, CNN)* involving different libraries *(SKLearn, Tensorflow)* and comparing the performance with and without GPU acceleration.

## Dependencies

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers
from sklearn.linear_model import LogisticRegression

readings = []

C:\Users\Hassan\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Hassan\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Hassan\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Hassan\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: P

## Preprocessing

In [2]:
(train_images, train_labels), (_i, _l) = mnist.load_data()

train_images = train_images.reshape(-1,28*28)
train_images = train_images / 255.0

## SieKitLearn - Logistic Regression

In [3]:
LG = LogisticRegression(penalty='l1', solver='saga', tol=0.1)

### sklearn timer

In [4]:
%%timeit -n 1 -r 10 -o
LG.fit(train_images, train_labels)

52.3 s ± 5.58 s per loop (mean ± std. dev. of 10 runs, 1 loop each)


<TimeitResult : 52.3 s ± 5.58 s per loop (mean ± std. dev. of 10 runs, 1 loop each)>

In [5]:
readings.append(_.all_runs)

[55.86617030000025,
 45.18151899999975,
 56.94857019999972,
 58.526647599999706,
 55.44186370000034,
 48.23873319999984,
 56.911804400000165,
 56.549121000000014,
 44.315490299999965,
 44.676526300000205]

## Tensorflow - ANN

In [6]:
annModel = keras.Sequential()
annModel.add(tf.keras.Input(shape=(28*28,)))
annModel.add(layers.Dense(128, activation='relu'))
annModel.add(layers.Dense(10, activation='softmax'))

annModel.compile('sgd','sparse_categorical_crossentropy',['accuracy'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


### ANN timer (CPU)

In [7]:
%%timeit -n 1 -r 10 -o
with tf.device('/CPU:0'):
    annModel.fit(train_images, train_labels, epochs=5, verbose=0)

17.8 s ± 989 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


<TimeitResult : 17.8 s ± 989 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)>

In [8]:
readings.append(_.all_runs)

[18.22845750000033,
 16.880923700000494,
 17.30925060000027,
 17.852184800000032,
 18.036119999999755,
 19.46307009999964,
 19.337056100000154,
 18.127447400000165,
 16.420312500000364,
 16.64615649999996]

### ANN timer (GPU)

In [ ]:
%%timeit -n 1 -r 10 -o
with tf.device('/GPU:0'):
    annModel.fit(train_images, train_labels, epochs=5, verbose=0)

In [ ]:
readings.append(_.all_runs)

## Tensorflow - CNN

In [9]:
cnnModel = keras.Sequential()
cnnModel.add(tf.keras.Input(shape=(28, 28, 1)))
cnnModel.add(layers.Conv2D(filters=16,kernel_size=(3, 3),activation='relu'))
cnnModel.add(layers.BatchNormalization())
cnnModel.add(layers.MaxPooling2D())
cnnModel.add(layers.Flatten())
cnnModel.add(layers.Dense(128, activation='relu'))
cnnModel.add(layers.Dropout(0.2))
cnnModel.add(layers.Dense(10, activation='softmax'))

cnnModel.compile('sgd','sparse_categorical_crossentropy',['accuracy'])

### CNN timer (CPU)

In [10]:
%%timeit -n 1 -r 10 -o
with tf.device('/CPU:0'):
    cnnModel.fit(train_images.reshape(-1, 28, 28, 1), train_labels, epochs=5, verbose=0)

6min 37s ± 16.6 s per loop (mean ± std. dev. of 10 runs, 1 loop each)


<TimeitResult : 6min 37s ± 16.6 s per loop (mean ± std. dev. of 10 runs, 1 loop each)>

In [11]:
readings.append(_.all_runs)

[389.2129934000004,
 404.6648078999997,
 410.26351650000015,
 414.05902589999914,
 418.9365496999999,
 402.0142014999992,
 405.5540261999995,
 387.22390819999964,
 379.7357458000006,
 362.37694459999966]

### CNN timer (GPU)

In [ ]:
%%timeit -n 1 -r 10 -o_.all_runs
with tf.device('/GPU:0'):
    cnnModel.fit(train_images.reshape(-1, 28, 28, 1), train_labels, epochs=5, verbose=0)

In [ ]:
readings.append(_.all_runs)

## Storing readings

In [12]:
import csv

with open('readings', 'w') as f:
    wr = csv.writer(f)
    wr.writerow(readings)

Done :)